In [17]:
from SPARQLWrapper import SPARQLWrapper, JSON
import re

In [4]:
sparql = SPARQLWrapper("http://localhost:9999/blazegraph/namespace/hogeraad/sparql")

In [2]:
def create_link(ecli_from, ecli_to, sparql):
    base_query = '''
    prefix dcterm: <http://purl.org/dc/terms/>
    insert data 
    {{<http://deeplink.rechtspraak.nl/uitspraak?id={}>
        dcterm:references 
        <http://deeplink.rechtspraak.nl/uitspraak?id={}> 
    .}}'''
    query = base_query.format(ecli_from, ecli_to)
    sparql.method = 'POST'
    sparql.setQuery(query)
    res = sparql.query()

In [14]:
query = '''
prefix dcterm: <http://purl.org/dc/terms/>
select ?id ?name
where {
    ?id dcterm:hasVersion ?name.
    filter (?name != "Rechtspraak.nl").
  }
'''

sparql.setQuery(query)
sparql.setReturnFormat(JSON)
ret = sparql.query()
id_name = ret.convert()

In [37]:
id_name['results']['bindings'][10]

{'id': {'type': 'uri',
  'value': 'http://deeplink.rechtspraak.nl/uitspraak?id=ECLI:NL:HR:2007:BA7217'},
 'name': {'type': 'literal',
  'value': 'JA 2007/157 met annotatie van W. Bouman'}}

In [44]:
len(id_name['results']['bindings'])

81208

In [45]:
regex = r'(.*)( met annotatie van .*$)'

aliases = {}

for item in id_name['results']['bindings']:
    id0 = item['id']['value']
    name = item['name']['value']
    # If the name contains an annotation, remove that one
    match = re.match(regex, name)
    if match is not None:
        name = match.groups()[0]
    
    aliases[name] = id0

In [50]:
list(aliases.keys())[:10]

['FutD 2013-2951',
 'V-N 1998/8.11',
 'RvdW 2010, 494',
 'NTFR 2000, 1849',
 'JOW 2003, 37',
 'BNB 2004/154',
 'NS 2008, 441',
 'SR-Updates.nl 2015-0478',
 'NJ 2007, 326',
 'NBSTRAF 2010/180']

In [40]:

match = re.match(regex, 'Jaea e 32351')
match.groups()

AttributeError: 'NoneType' object has no attribute 'groups'

In [30]:
regex = r'(.*)(met annotatie van W. Bouman)'
match = re.match(regex, ex_name)

('JA 2007/157 ', 'met annotatie van W. Bouman')